<a href="https://colab.research.google.com/github/janaat22/NLP_NER/blob/master/NER_Model_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')


In [82]:
pip install sklearn-crfsuite


In [83]:
pip install nltk

In [0]:
import nltk

In [85]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
nltk.download('gazetteers')

[nltk_data] Downloading package gazetteers to /root/nltk_data...
[nltk_data]   Package gazetteers is already up-to-date!


True

In [87]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [0]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
#from sklearn_crfsuite import CRF, scorers, metrics
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer

from nltk.stem.porter import *
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import names, ieer, gazetteers

In [0]:
locations = gazetteers.words()
locations = [x.lower() for x in locations]
names_list = names.words()
names_list = [x.lower() for x in names_list]

In [90]:
word = 'LA'
word.lower() in locations

True

In [91]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:


def file_to_list(file_name):  
  training_key_value = []
  word_dict = {}
  tags = {}
  word_list = []
  sentence = 1
  file1 = open(file_name, 'r')
  Lines = file1.readlines() 
  for line in Lines:    
    training_key_value = line.split()
    if(len(training_key_value) == 2):
      word = training_key_value[0]
      tag = training_key_value[1]
      word_dict[word] = tag
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      word_list.append(([sentence, word, tagged[0][1], tag])) 
      tags.update(tagged)
    else:
      sentence = sentence + 1
    file1.close()
  return word_dict,word_list

#Train data
word_dict_train, word_list_train = file_to_list('train.txt')
# print(word_dict_train.keys())
# array_of_words_train = np.array(list(word_dict_train.keys()))
# array_of_values_train = np.array(list(word_dict_train.values()))
# X_Y_train = np.column_stack((array_of_words_train, array_of_values_train))

#Dev data
word_dict_dev,word_list_dev = file_to_list('dev.txt')
# array_of_words_dev = np.array(list(word_dict_dev.keys()))
# array_of_values_dev = np.array(list(word_dict_dev.values()))
# X_Y_dev = np.column_stack((array_of_words_dev, array_of_values_dev))




In [0]:
def file_to_list_test(file_name):  
  training_key_value = []
  word_dict = {}  
  word_list = []
  training_key_value = [] 
  test_word_list = []
  file1 = open(file_name, 'r')
  sentence_num = 1
  Lines = file1.readlines() 
  # print("print" + str(len(Lines[11].split('\n'))))
  for line in Lines:    
    if(len(line) > 1):      
      word = line.replace('\n', '')
      tokenized = nltk.word_tokenize(word)
      tagged = nltk.pos_tag(tokenized)
      # print(sentence_num)
      test_word_list.append(([sentence_num, word, tagged[0][1]]))
    else:
      # print("\nelse")
      sentence_num = sentence_num + 1
  file1.close()
  return test_word_list



#Test data
word_list_test = file_to_list_test('test_no_tag.txt')


In [94]:
train_data = pd.DataFrame(word_list_train)
print(train_data.head(n = 4))

dev_data = pd.DataFrame(word_list_dev)
print(dev_data.head(n = 4))


test_data = pd.DataFrame(word_list_test)
print(test_data.head(n = 4))
# print(word_list_test)

   0                1    2  3
0  1  @SammieLynnsMom   NN  O
1  1         @tg10781   NN  O
2  1             they  PRP  O
3  1             will   MD  O
   0       1   2  3
0  1    STOP  NN  O
1  1    WHAT  WP  O
2  1  YOU'RE  NN  O
3  1   DOING  NN  O
   0       1    2
0  1      We  PRP
1  1     're  VBP
2  1  slaves  NNS
3  1      to   TO


In [95]:
# train_data.columns = train_data.iloc[0]
# train_data = train_data[1:]
train_data.columns = ['Sentence','Word','POS','Tag']
# train_data = train_data.reset_index(drop=True)
#train_data.head(n=5)

# dev_data.columns = dev_data.iloc[0]
# dev_data = dev_data[1:]
dev_data.columns = ['Sentence','Word','POS','Tag']
# dev_data = dev_data.reset_index(drop=True)
#dev_data.head(n=5)

# test_data.columns = test_data.iloc[0]
# test_data = test_data[1:]
test_data.columns = ['Sentence','Word','POS']
# test_data = test_data.reset_index(drop=True)
test_data.head(n= 10)


,Sentence,Word,POS
0,1,We,PRP
1,1,'re,VBP
2,1,slaves,NNS
3,1,to,TO
4,1,a,DT
5,1,world,NN
6,1,we,PRP
7,1,mastered,VBN
8,1,.,.
9,2,I,PRP


In [96]:
test_data.shape

(42413, 3)

In [0]:
# A class to retrieve the sentences from the dataset
class getsentence(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

class getsentence_ForTest(object):
    
    def __init__(self, data):
        self.n_sent = 1.0
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p) for w, p in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist())
                                                      ]
        self.grouped = self.data.groupby("Sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [98]:
train_getter = getsentence(train_data)
train_sentences = train_getter.sentences

dev_getter = getsentence(dev_data)
dev_sentences = dev_getter.sentences

test_getter = getsentence_ForTest(test_data)
test_sentences = test_getter.sentences

#This is how a sentence will look like. 
print(train_sentences[0])

[('@SammieLynnsMom', 'NN', 'O'), ('@tg10781', 'NN', 'O'), ('they', 'PRP', 'O'), ('will', 'MD', 'O'), ('be', 'VB', 'O'), ('all', 'DT', 'O'), ('done', 'VBN', 'O'), ('by', 'IN', 'O'), ('Sunday', 'NNP', 'O'), ('trust', 'NN', 'O'), ('me', 'PRP', 'O'), ('*wink*', 'NN', 'O')]


In [0]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
        'inLocations': word.lower() in locations if not word.find('#') else word.lower().replace('#','') in locations,
        'inNames' : word.lower() in names_list if not word.find('@') else word.lower().replace('@','') in names_list,
        'isTag' : word.find('@'),
        'hasCapitalLetter' : word[0].isupper(),
        # 'stem' : stemmer.stem(word),        
        'lemma' : lemmatizer.lemmatize(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
            '-1:inLocations': word1.lower() in locations if not word1.find('#') else word1.lower().replace('#','') in locations,
            '-1:inNames' : word1.lower() in names_list if not word1.find('@') else word1.lower().replace('@','') in names_list,
            '-1:hasCapitalLetter' : word1[0].isupper(),
            # '-1:stem' : stemmer.stem(word1),            
            '-1:lemma' : lemmatizer.lemmatize(word1),
             
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
            '+1:inLocations': word1.lower() in locations if not word1.find('#') else word1.lower().replace('#','') in locations,
            '+1:inNames' :word1.lower() in names_list if not word1.find('@') else word1.lower().replace('@','') in names_list,
            '+1:hasCapitalLetter' : word1[0].isupper(),
            # '+1:stem' : stemmer.stem(word1),            
            '+1:lemma' : lemmatizer.lemmatize(word),
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [100]:
print(test_sentences[5])
sent2features(train_sentences[3])[0]
# sent2features(test_sentences[7])[0]

[('Happy', 'JJ'), ('v-day', 'NN'), ('😌', 'NN'), ('❤', 'NN'), ('️', 'NN'), ('😂😂😂😂', 'NN'), ('#noshame', '#')]


{'+1:hasCapitalLetter': False,
 '+1:inLocations': False,
 '+1:inNames': False,
 '+1:lemma': '@ls_n',
 '+1:postag': 'RB',
 '+1:postag[:2]': 'RB',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'perhaps',
 'BOS': True,
 'bias': 1.0,
 'hasCapitalLetter': False,
 'inLocations': False,
 'inNames': False,
 'isTag': 0,
 'lemma': '@ls_n',
 'postag': 'NN',
 'postag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': False,
 'word.isupper()': False,
 'word.lower()': '@ls_n',
 'word[-2:]': '_n',
 'word[-3:]': 's_n'}

In [0]:
#Creating the train and test set

X_train = [sent2features(s) for s in train_sentences]
y_train = [sent2labels(s) for s in train_sentences]

X_dev = [sent2features(s) for s in dev_sentences]
y_dev = [sent2labels(s) for s in dev_sentences]

X_test = [sent2features(s) for s in test_sentences]


In [0]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
crf = crf.fit(X_train,y_train)

In [104]:
pred = cross_val_predict(estimator=crf, X = X_train,y = y_train, cv=10)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [105]:
#evaluate the model
report = flat_classification_report(y_pred=pred, y_true=y_train)
print(report)
# f1 = metrics.flat_f1_score(y_train, y_pred, average='weighted', labels=labels)
# print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company       0.87      0.46      0.60       171
     B-group       0.67      0.19      0.29       106
  B-location       0.77      0.54      0.63       380
     B-other       0.53      0.24      0.33       225
    B-person       0.79      0.64      0.71       449
   B-product       0.79      0.20      0.31        97
     B-title       0.54      0.22      0.31        68
   I-company       0.57      0.11      0.19        36
     I-group       0.52      0.14      0.22        84
  I-location       0.59      0.36      0.45       154
     I-other       0.47      0.29      0.36       320
    I-person       0.78      0.71      0.74       215
   I-product       0.70      0.17      0.28        80
     I-title       0.46      0.21      0.29        77
           O       0.97      0.99      0.98     44007

    accuracy                           0.96     46469
   macro avg       0.67      0.37      0.45     46469
weighted avg       0.96   

In [106]:
# labels = list(crf.classes_)

# labels.remove('O')
print(labels)


['B-location', 'I-location', 'B-title', 'I-title', 'B-company', 'B-product', 'B-person', 'B-other', 'I-other', 'B-group', 'I-group', 'I-product', 'I-company', 'I-person']


In [107]:
y_pred = crf.predict(X_dev)
metrics.flat_f1_score(y_dev, y_pred,
                      average='weighted', labels=labels)

0.31361687449049797

In [108]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("Accuracy: " + str(accuracy))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))

recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
print("F1 weighted avg: " + str(f1))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   B-company      0.462     0.154     0.231        39
   I-company      0.000     0.000     0.000        10
     B-group      0.500     0.045     0.083       111
     I-group      0.143     0.021     0.036        48
  B-location      0.612     0.461     0.526       154
  I-location      0.557     0.420     0.479        81
     B-other      0.462     0.182     0.261       132
     I-other      0.188     0.289     0.228        97
    B-person      0.624     0.515     0.564       171
    I-person      0.588     0.600     0.594        95
   B-product      1.000     0.027     0.053        37
   I-product      0.000     0.000     0.000       121
     B-title      0.167     0.059     0.087        17
     I-title      0.111     0.067     0.083        15

   micro avg      0.476     0.281     0.353      1128
   macro avg      0.387     0.203     0.230      1128
weighted avg      0.446     0.281     0.314      1128

Accuracy: 0.9406555562388

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Hyper Parameter Optimization

In [109]:
Model_X = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 35.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=False, averaging=None,
                                 c=None, c1=0.1, c2=0.1,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_t...
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7faa4c5d6d30>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False,
                   scoring=make_scorer(flat_f1_score, average=weighted, l

# Best Parameters

In [110]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.13208146892209183, 'c2': 0.003605084094505999}
best CV score: 0.46883084031421807
model size: 0.73M


In [111]:
crf = rs.best_estimator_
y_pred = crf.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
print("accuracy :" + str(accuracy))
precision = metrics.flat_precision_score(y_dev, y_pred, average='weighted', labels=labels)
print("Precision : "  + str(precision))
recall = metrics.flat_recall_score(y_dev, y_pred, average='weighted', labels=labels)
print("Recall : " + str(recall))
f1 = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
print("F1 weighted avg: " + str(f1))

              precision    recall  f1-score   support

   B-company      0.500     0.154     0.235        39
   I-company      0.000     0.000     0.000        10
     B-group      0.500     0.054     0.098       111
     I-group      0.200     0.021     0.038        48
  B-location      0.568     0.461     0.509       154
  I-location      0.571     0.444     0.500        81
     B-other      0.438     0.212     0.286       132
     I-other      0.181     0.309     0.228        97
    B-person      0.537     0.515     0.525       171
    I-person      0.558     0.611     0.583        95
   B-product      1.000     0.108     0.195        37
   I-product      1.000     0.017     0.033       121
     B-title      0.000     0.000     0.000        17
     I-title      0.000     0.000     0.000        15

   micro avg      0.446     0.293     0.353      1128
   macro avg      0.432     0.208     0.231      1128
weighted avg      0.529     0.293     0.316      1128

accuracy :0.9379496955906

In [112]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-group -> I-group 9.396314
B-title -> I-title 9.289197
B-product -> I-product 8.864227
B-person -> I-person 8.337055
I-group -> I-group 7.989319
I-title -> I-title 7.976070
B-other -> I-other 7.774106
B-location -> I-location 7.661786
I-product -> I-product 7.482547
B-company -> I-company 7.256505
I-company -> I-company 6.725760
I-other -> I-other 6.361689
I-location -> I-location 5.916330
I-person -> I-person 5.322534
O      -> O       4.013160
O      -> B-person 2.490924
O      -> B-company 2.432572
O      -> B-product 2.225317
O      -> B-title 1.883259
O      -> B-group 1.780581

Top unlikely transitions:
B-company -> O       0.327161
B-person -> O       0.289483
B-group -> O       0.157608
I-person -> O       0.072233
I-company -> B-location 0.064935
B-product -> O       0.021644
I-location -> B-group 0.019760
I-title -> O       -0.150863
B-other -> O       -0.159177
B-location -> B-location -0.177735
I-group -> O       -0.192949
I-product -> O       -0.23

In [113]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
12.179724 B-company word.lower():twitter
10.221582 B-product word.lower():ipod
9.816257 B-company word.lower():facebook
8.817285 B-person word.lower():pope
7.030279 I-other  -1:word.lower():fashion
6.752532 B-other  word[-2:]:BL
6.572125 B-product word.lower():xbox
6.314450 B-product word.lower():ipad
5.756496 B-group  +1:word.lower():gear
5.681476 B-other  word[-2:]:GP
5.619877 B-company word.lower():youtube
5.555898 B-person word.lower():eminem
5.487478 B-other  word[-3:]:mas
5.269280 B-location word[-3:]:ham
5.047971 O        +1:word.lower():bless
5.008578 B-person word.lower():taylor
4.901854 B-group  -1:lemma:Go
4.868173 O        EOS
4.583040 I-location word.lower():lounge
4.522804 B-location inLocations
4.502677 B-company +1:word.lower():instant
4.380057 B-person word[-3:]:nem
4.376621 B-location +1:word.lower():tall
4.269842 I-other  word.lower():festival
4.265163 O        bias
4.253721 B-location +1:word.lower():winds
4.194503 B-location word.lower():uk
3.973192 B

In [114]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(y_pred[0][0])

O


In [0]:
outR = open("test_no_tag.txt", "r")
outF = open("result.txt", "w")
lines = outR.readlines()
pred_line = 0
pred_word = 0
for line in lines:
  if(len(line)>1):
    outF.write(line.replace('\n','')+"\t\t"+y_pred[pred_line][pred_word]+"\n")
    pred_word = pred_word + 1
  else:
    outF.write("\n")
    pred_word = 0
    pred_line = pred_line + 1


outR.close()
outF.close()